# Run obstacle case by PyRunOF. 

The test was created to explain the general options and possibility of the python libraries PyRunOF. 

The test consist of 
    
    * settings folder is the folder to be stored a setting to run simulation.
    * solution folder is the folder to be stored solution of simulation.
    * compute.py is the general python script to run the case in parametric sweep mode.
    * test_lib.ipynb is the python notebook considering procedures of simulation of obstacle case.
    * test_lib.py is the python script to run simulation of obstacle case.

Let's start to import pyRunOF libraries and input data from data.py in settings folder. 
The data.py consist of parameters describing the modeling system. For example, geometry, mesh and other parameters. 

In [9]:
import pyRunOF
from settings.data import *

To make easy 

Initialize a class to process case. Here we declare path of working folder

In [2]:

mp = pyRunOF.Manipulations(dir_path=dir_path)

## Set path for folder with settings and solution
mp.create_path_dir(dir_path_key='dir', case_name='settings',
                    path_key='settings')
mp.create_path_dir(dir_path_key='dir', case_name='solution',
                    path_key='solution')

# set path of source case
mp.create_name(name_base=src_case, name_key=src_name_key)
mp.create_path_dir(dir_path_key='settings', name_key=src_name_key,
                    path_key=src_path_key)
# path to json parameters
mp.create_path_dir(dir_path_key='settings', case_name='parameters.json',
                    path_key='parameters_path')
# path to json parameters of mesh
mp.create_path_dir(dir_path_key='settings', case_name='mesh_parameters.json',
                    path_key='mesh_parameters_path')
# имя скрипта для создания сетки
mp.create_name(name_base='create_obstacle_mesh.py', only_base=True, name_key='salome_script')
# путь к скрипту сетки
mp.create_path_dir(dir_path_key='settings', name_key='salome_script', path_key='salome_script_path')

# set path to general parameters
mp.create_path_dir(dir_path_key='settings', case_name='parameters.json',
                    path_key='parameters_path')
# set path to general parameters
mp.create_path_dir(dir_path_key='settings', case_name='mesh_parameters.json',
                    path_key='mesh_parameters_path')

# collect general dict of parameters и записывает в путь по ключу parameters_path
mp.create_json_params(time_dict, parallel_dict, prop_dict, zero_dict, library_path,
                        mp.get_dict_from_json(mp.get_path('mesh_parameters_path')),
                        save_path=mp.get_path('parameters_path'))

# if the solution folder does not exist we create it by below written code
if not os.path.exists(mp.get_path('solution')):
    mp.create_folder(dir_key='dir', folder_name='solution')

In [3]:
mp.create_name('test', name_base=src_case, name_key=dst_name_key)
mp.create_path_dir(dir_path_key='solution', name_key=dst_name_key,
                    path_key=dst_path_key)

mp.duplicate_case(src_key=src_path_key, dist_key=dst_path_key, mode='rewrite')




In [4]:
data = mp.get_dict_from_json(mp.get_path('parameters_path'))

# here update velocity according Re number
# data['U_var'] = calculate_velocity(data['Re_var'], data['nu_var'], data['d_var'])
# mp.change_json_params(mp.get_path('mesh_parameters_path'), data)

########### System folder #######################
system = pyRunOF.System(case_path=mp.get_path(dst_path_key))
system.set_controlDict(data) #, case_path=mp.get_path(dst_path_key))


In [5]:
########### Constant folder #####################
constant = pyRunOF.Constant(case_path=mp.get_path(dst_path_key), lib_path=library_path['lib_path_var'])
constant.set_transportProp(data)
"""
LES - Large eddy simulation
            kEpsilon
            realizablekE
            kOmega
            kOmegaSST
            laminar
            LESdynamicSmag
"""

constant.turbulent_model(turbulent_type='LESdynamicSmag')  # , case_path=mp.get_path(dst_path_key))



In [6]:
########### Initial conditions #####################
zero = pyRunOF.InitialValue(case_path=mp.get_path(dst_path_key))

data.update(zero.calcInitVal_cylindr(data['d_var'], data['U_var'], data['nu_var']))

zero.set_var(data, file_names=zero.find_all_zero_files())
                #case_path=mp.get_path(dst_path_key),
                #)

########### mesh settings #####################
mesh = pyRunOF.Mesh(case_path=mp.get_path(dst_path_key))
mesh.set_decomposePar(data)

mp.create_name(name_base='create_obstacle_mesh.py', only_base=True, name_key='salome_script')
mp.create_path_dir(dir_path_key='settings', name_key='salome_script', path_key='salome_script_path')
poly_mesh_path = mp.get_constant_path(str(mp.get_path(dst_path_key))) / 'polyMesh'

data.update({'constant_path': str(poly_mesh_path)})

mp.create_path_dir(dir_path_key='settings', case_name='parameters.json',
                    path_key='parameters_path_new')

# collect general dict of parameters и записывает в путь по ключу parameters_path
mp.create_json_params(data, save_path=mp.get_path('parameters_path_new'))


In [7]:
mesh.run_salome_mesh(script_path=mp.get_path('salome_script_path'),
                        parameter_path=mp.get_path('parameters_path_new'))



bash: cannot set terminal process group (24824): Inappropriate ioctl for device
bash: no job control in this shell


/home/ivan/programms/OpenFOAM/PyRunOF/tests/obstacle_new_ps/settings
"ViscousLayers2D=0.00086...,7,1.03,[10, 3],0" was not assigned to "final geometry" shape in "final geometry" mesh  : hypothesis of the same dimension is already assigned to this shape
Number of nodes: 51896

Number of cells: 25475

Counting number of faces:

total number of faces: 102373, internal: 50355, external 52018

Converting mesh to OpenFOAM

Finished processing boundary faces

Finished processing volumes.

Sorting faces in upper triangular order

Writing the file points

Writing the file faces

Writing the file owner

Writing the file neighbour

Writing the file boundary

Writing file cellZones

Finished writing to /home/ivan/programms/OpenFOAM/PyRunOF/tests/obstacle_new_ps/settings//home/ivan/programms/OpenFOAM/PyRunOF/tests/obstacle_new_ps/solution/obstacle_base_test/constant/polyMesh 

Converted mesh in 2s

Wrote mesh in 1s

Total time: 3s


In [8]:
mesh.run_decompose(what='OF')

bash: cannot set terminal process group (24824): Inappropriate ioctl for device
bash: no job control in this shell


/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  6
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
Build  : 6-47517f2ebb1b
Exec   : decomposePar -force
Date   : Jan 25 2024
Time   : 14:35:50
Host   : "ivan"
PID    : 25227
I/O    : uncollated
Case   : /home/ivan/programms/OpenFOAM/PyRunOF/tests/obstacle_new_ps/solution/obstacle_base_test
nProcs : 1
sigFpe : Enabling floating point exception trapping (FOAM_SIGFPE).
fileModificationChecking : Monitoring run-time modified files using timeStampMaster (fileModificationSkew 10)
allowSystemOperations : Allowing user-supplied system call operations

// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //
Create time

Removing 0 existi

In [ ]:
runner = pyRunOF.Run(solver='pisoFoam', path_case=mp.get_path(dst_path_key))
runner.set_log_flag(log_flag=True)
runner.set_mode(mode='parallel')
runner.set_cores(coreOF=data['core_OF'])
runner.run()
